<a href="https://colab.research.google.com/github/OneFineStarstuff/Onefinebot/blob/main/Text_to_Image_Generation_Using_Diffusion_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

class SimpleDiffusionModel(nn.Module):
    def __init__(self, img_size, channels):
        super(SimpleDiffusionModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(channels, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, channels, 4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

def load_image(path, img_size):
    img = Image.open(path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
    img = transform(img).unsqueeze(0)
    return img

def save_image(tensor, path):
    img = tensor.cpu().clone().squeeze(0)
    img = transforms.ToPILImage()(img)
    img.save(path)

def train_model(model, dataloader, num_epochs, lr=0.001):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    model.train()

    for epoch in range(num_epochs):
        for imgs in dataloader:
            optimizer.zero_grad()
            output = model(imgs)
            loss = criterion(output, imgs)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

def generate_image(model, text_embedding, img_size, num_steps=1000):
    noise = torch.randn(1, 3, img_size, img_size)
    noise = noise.to(next(model.parameters()).device)
    for step in range(num_steps):
        noise.requires_grad_(True)
        output = model(noise)
        noise = noise - 0.1 * torch.autograd.grad((output - text_embedding).abs().sum(), noise)[0]
    return output

# Example usage
img_size = 64
channels = 3
model = SimpleDiffusionModel(img_size, channels)
model = model.to("cuda" if torch.cuda.is_available() else "cpu")

# Load an image and use it as a stand-in for text embedding
input_img = load_image("path_to_image.jpg", img_size)
input_img = input_img.to(next(model.parameters()).device)

# Train model (you'd normally use a proper dataset and DataLoader here)
dataloader = [input_img] * 100  # Simplified example with repeated image
train_model(model, dataloader, num_epochs=5)

# Generate new image based on text embedding
generated_img = generate_image(model, input_img, img_size)
save_image(generated_img, "generated_image.png")